In [5]:
!pip install numpy pandas scikit-learn matplotlib tqdm scipy


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 370.8 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/8.1 MB 370.8 kB/s eta 0:00:21
   -- ------------------------------------- 0.5/8.1 MB 370.8 kB/s eta 0:00:21
   --- ------------------------------------ 0.8/8.1 MB 389.2 kB/s eta 0:00:19
   ----- ---------------------------------- 1.0/8.1 MB 508.7 kB/s eta 0:00:

In [ ]:
# q3_complete.py
# Complete code for Assignment Q3 (parts a and b)
# Requires: scikit-learn, numpy, pandas, matplotlib, tqdm

import os
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_kddcup99
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.sparse as sp

# ---------------------
# Utilities
# ---------------------
def ensure_dir(d):
    os.makedirs(d, exist_ok=True)

def load_and_preprocess_kdd(subsample=None, scale=True):
    """
    Fetch KDDCup99, decode bytes, label-encode categorical columns, return numeric D and y.
    If subsample is int, take first `subsample` rows for faster experiments.
    """
    data = fetch_kddcup99(subset=None, download_if_missing=True)
    X_raw = np.array(data.data)  # shape (n, d) but entries are bytes or numbers
    y_raw = np.array(data.target)

    n, d = X_raw.shape
    print(f"Raw dataset shape: n={n}, d={d}")

    # decode / convert column-wise
    X_cols = []
    for j in range(d):
        col = X_raw[:, j]
        # detect bytes
        if isinstance(col[0], (bytes, bytearray)):
            # decode and label encode
            col_decoded = [c.decode('utf-8', errors='ignore') for c in col]
            le = LabelEncoder()
            col_enc = le.fit_transform(col_decoded)
            X_cols.append(col_enc.astype(np.float32))
        else:
            X_cols.append(col.astype(np.float32))

    X = np.vstack(X_cols).T  # shape (n, d)
    # y: decode and label encode
    y_decoded = [v.decode('utf-8', errors='ignore') for v in y_raw]
    le_y = LabelEncoder()
    y_enc = le_y.fit_transform(y_decoded).astype(np.float32)

    if subsample is not None:
        X = X[:subsample]
        y_enc = y_enc[:subsample]

    if scale:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

    print(f"Processed X shape: {X.shape}, y shape: {y_enc.shape}")
    return X, y_enc

# ---------------------
# JL matrix factories
# ---------------------
def dense_gaussian_jl(d, x, rng=None):
    """
    Return dense Gaussian JL matrix of shape (d, x) such that columns scaled by 1/sqrt(x).
    So E = D @ M gives shape (n, x).
    """
    if rng is None:
        rng = np.random.default_rng()
    M = rng.normal(loc=0.0, scale=1.0/np.sqrt(x), size=(d, x)).astype(np.float32)
    return M

def sparse_achlioptas_jl(d, x, s=3, rng=None):
    """
    Achlioptas-style sparse JL: each entry is:
      +sqrt(s) with prob 1/(2s)
      -sqrt(s) with prob 1/(2s)
      0 with prob 1 - 1/s
    M has shape (d, x)
    """
    if rng is None:
        rng = np.random.default_rng()
    prob = 1.0 / s
    size = d * x
    # generate in chunks if needed
    vals = rng.random(size=size)
    M = np.zeros(size, dtype=np.float32)
    pos = (vals < (prob/2))
    neg = (vals >= (prob/2)) & (vals < prob)
    M[pos] = np.sqrt(s)
    M[neg] = -np.sqrt(s)
    M = M.reshape((d, x))
    return M

# ---------------------
# Part 3(a): JL + k-means experiments
# ---------------------
def compute_kmeans_cost(D, centroids):
    # vectorized: for each point find squared distance to nearest centroid
    # D: (n, d), centroids: (k, d)
    # compute squared norms
    # use broadcasting carefully to avoid memory blowup
    n, d = D.shape
    k = centroids.shape[0]
    # we'll compute in chunks if n big
    chunk = 10000
    cost = 0.0
    for i in range(0, n, chunk):
        Di = D[i:i+chunk]  # (c, d)
        # distances: (c, k) = sum((Di[:,None,:] - centroids[None,:,:])**2, axis=2)
        # compute efficiently: ||u-v||^2 = ||u||^2 + ||v||^2 - 2 u.v
        u2 = np.sum(Di**2, axis=1)[:, None]  # (c,1)
        v2 = np.sum(centroids**2, axis=1)[None, :]  # (1,k)
        uv = Di @ centroids.T  # (c,k)
        dists2 = u2 + v2 - 2*uv
        cost += np.sum(np.min(dists2, axis=1))
    return float(cost)

def part3a_experiment(D, x_values=[5,15,20,25], trials=5, kmeans_k=15, rng_seed=0, use_sparse=False):
    rng = np.random.default_rng(rng_seed)
    n, d = D.shape
    results = {}
    # baseline: kmeans on original D => B
    print("Running baseline kmeans on original data (this may take time)...")
    km_full = KMeans(n_clusters=kmeans_k, random_state=rng_seed, n_init=10).fit(D)
    B = km_full.cluster_centers_
    cost_B = compute_kmeans_cost(D, B)
    results['baseline_cost'] = cost_B
    print(f"Baseline cost (kmeans on D): {cost_B:.3f}")

    for x in x_values:
        costs_using_A = []
        costs_using_B_as_control = []
        for t in range(trials):
            if use_sparse:
                M = sparse_achlioptas_jl(d, x, s=3, rng=rng)
            else:
                M = dense_gaussian_jl(d, x, rng=rng)
            E = D @ M  # n x x
            # run kmeans on projected data
            km_proj = KMeans(n_clusters=kmeans_k, random_state=int(rng.integers(0,1e9)), n_init=10).fit(E)
            A_proj = km_proj.cluster_centers_  # shape (k, x)
            # map centroids back to original space: A_back = A_proj @ M.T  (k x d)
            A_back = A_proj @ M.T
            cost_using_A = compute_kmeans_cost(D, A_back)
            costs_using_A.append(cost_using_A)
            # also compute cost of using centroids B (from D) on D as control (should equal baseline)
            costs_using_B_as_control.append(cost_B)
            print(f"x={x} trial {t+1}/{trials}: cost_using_A={cost_using_A:.3f}")
        results[x] = {'costs_using_A': costs_using_A, 'mean_cost': float(np.mean(costs_using_A))}
    return results

# ---------------------
# Part 3(b): Sparse JL + linear regression
# ---------------------
def sparse_jl_matrix_x_n(x, n, s=3, rng=None):
    """
    Return dense-ish sparse JL of shape (x, n) using Achlioptas style,
    implemented as dense numpy for simplicity. For very large n use scipy.sparse.
    """
    if rng is None:
        rng = np.random.default_rng()
    prob = 1.0 / s
    size = x * n
    vals = rng.random(size=size)
    Mflat = np.zeros(size, dtype=np.float32)
    pos = (vals < (prob/2))
    neg = (vals >= (prob/2)) & (vals < prob)
    Mflat[pos] = np.sqrt(s)
    Mflat[neg] = -np.sqrt(s)
    M = Mflat.reshape((x, n))
    return M

def part3b_experiment(D, y, x_values=[25,50,100,200,500], trials=5, alpha=1.0, rng_seed=0):
    rng = np.random.default_rng(rng_seed)
    n, d = D.shape
    results = {}
    # Fit baseline ridge on (D,y)
    print("Fitting baseline Ridge on (D,y)...")
    ridge_full = Ridge(alpha=alpha, fit_intercept=True)
    ridge_full.fit(D, y)
    b = ridge_full.coef_
    loss_full = float(np.linalg.norm(D.dot(b) - y)**2)
    results['baseline_loss'] = loss_full
    print(f"Baseline loss on (D,y): {loss_full:.3f}")

    for x in x_values:
        losses = []
        for t in range(trials):
            M = sparse_jl_matrix_x_n(x, n, s=3, rng=rng)  # shape (x, n)
            E = M @ D         # shape (x, d)
            z = M @ y         # shape (x,)
            # Solve ridge on (E, z): we want a in R^d
            model_proj = Ridge(alpha=alpha, fit_intercept=True)
            model_proj.fit(E, z)
            a = model_proj.coef_
            loss_using_a = float(np.linalg.norm(D.dot(a) - y)**2)
            losses.append(loss_using_a)
            print(f"x={x} trial {t+1}/{trials}: loss_using_a={loss_using_a:.3f}")
        results[x] = {'losses': losses, 'mean_loss': float(np.mean(losses))}
    return results

# ---------------------
# Runner
# ---------------------
def run_all_q3(subsample=None, outdir="./q3_outputs", trials=5):
    ensure_dir(outdir)
    D, y = load_and_preprocess_kdd(subsample=subsample)
    # Part 3a
    x_vals_a = [5, 15, 20, 25]
    print("\n=== Running Part 3a experiments ===")
    res3a = part3a_experiment(D, x_values=x_vals_a, trials=trials, kmeans_k=15, rng_seed=0, use_sparse=False)
    # Save results
    pd.DataFrame({x: res3a[x]['costs_using_A'] for x in x_vals_a}).to_csv(os.path.join(outdir, "part3a_costs.csv"), index=False)
    # plot boxplot of costs per x
    fig, ax = plt.subplots(figsize=(8,5))
    data = [res3a[x]['costs_using_A'] for x in x_vals_a]
    ax.boxplot(data, labels=[str(x) for x in x_vals_a])
    ax.set_xlabel("x (projected dim)")
    ax.set_ylabel("k-means cost on D using centroids from projected kmeans")
    ax.set_title("Part 3a: Cost of projected centroids mapped back to original space")
    fig.savefig(os.path.join(outdir, "part3a_costs_boxplot.png"))
    plt.close(fig)

    # Part 3b
    x_vals_b = [25, 50, 100, 200, 500]
    print("\n=== Running Part 3b experiments ===")
    res3b = part3b_experiment(D, y, x_values=x_vals_b, trials=trials, alpha=1.0, rng_seed=0)
    pd.DataFrame({x: res3b[x]['losses'] for x in x_vals_b}).to_csv(os.path.join(outdir, "part3b_losses.csv"), index=False)
    # plot
    fig, ax = plt.subplots(figsize=(8,5))
    data2 = [res3b[x]['losses'] for x in x_vals_b]
    ax.boxplot(data2, labels=[str(x) for x in x_vals_b])
    ax.set_xlabel("x (projected dimension)")
    ax.set_ylabel("Regression loss on (D,y) using coefficients learned from projected data")
    ax.set_title("Part 3b: Regression loss vs projected dimension x")
    fig.savefig(os.path.join(outdir, "part3b_losses_boxplot.png"))
    plt.close(fig)

    # Save summary JSON-like
    summary = {"part3a": {x: {'mean_cost': res3a[x]['mean_cost']} for x in x_vals_a},
               "part3a_baseline_cost": res3a['baseline_cost'],
               "part3b": {x: {'mean_loss': res3b[x]['mean_loss']} for x in x_vals_b},
               "part3b_baseline_loss": res3b['baseline_loss']}
    pd.DataFrame(summary).to_csv(os.path.join(outdir, "q3_summary.csv"), index=False)
    print(f"\nAll outputs saved in {outdir}")
    return res3a, res3b

if __name__ == "__main__":
    # set subsample=None to use full dataset; for quick tests set subsample=50000 or similar
    run_all_q3(subsample=None, outdir="./q3_outputs", trials=5)


Matplotlib is building the font cache; this may take a moment.


Raw dataset shape: n=494021, d=41
Processed X shape: (494021, 41), y shape: (494021,)

=== Running Part 3a experiments ===
Running baseline kmeans on original data (this may take time)...
Baseline cost (kmeans on D): 4562715.000
x=5 trial 1/5: cost_using_A=45439996.000
x=5 trial 2/5: cost_using_A=50483496.000
x=5 trial 3/5: cost_using_A=29748834.000
x=5 trial 4/5: cost_using_A=40004772.000
x=5 trial 5/5: cost_using_A=26811346.000
x=15 trial 1/5: cost_using_A=39726924.000
x=15 trial 2/5: cost_using_A=37089848.000
x=15 trial 3/5: cost_using_A=28853278.000
x=15 trial 4/5: cost_using_A=31391292.000
x=15 trial 5/5: cost_using_A=23356988.000
x=20 trial 1/5: cost_using_A=21286552.000
x=20 trial 2/5: cost_using_A=23176650.000
x=20 trial 3/5: cost_using_A=23571390.000
x=20 trial 4/5: cost_using_A=25602946.000
x=20 trial 5/5: cost_using_A=22272438.000
x=25 trial 1/5: cost_using_A=29618156.000
x=25 trial 2/5: cost_using_A=22549988.000
x=25 trial 3/5: cost_using_A=22254976.000
x=25 trial 4/5: cost

C:\Users\lenovo\AppData\Local\Temp\ipykernel_12904\3236362109.py:223: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=[str(x) for x in x_vals_a])



=== Running Part 3b experiments ===
Fitting baseline Ridge on (D,y)...
Baseline loss on (D,y): 104941976.000


C:\Users\lenovo\miniconda3\Lib\site-packages\scipy\_lib\_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=2.82216e-09): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)


...

C:\Users\lenovo\miniconda3\Lib\site-packages\scipy\_lib\_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=2.78069e-10): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)


x=500 trial 4/5: loss_using_a=113108624.000


C:\Users\lenovo\miniconda3\Lib\site-packages\scipy\_lib\_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=2.7769e-10): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)


x=500 trial 5/5: loss_using_a=114824512.000


C:\Users\lenovo\AppData\Local\Temp\ipykernel_12904\3236362109.py:238: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data2, labels=[str(x) for x in x_vals_b])
